In [1]:
import sys
from pathlib import Path
from os import path, chdir
from collections import defaultdict
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
import shutil


sys.path.append('..')

from utils.model_factory import load_models
from utils.data_holder import DataHolder

chdir('..')

In [3]:
models = load_models(Path('results', 'polynomial'))

for path, model in models:
    output_dir = path.parent / 'pdfs'
    if output_dir.exists():
        shutil.rmtree(output_dir)
    print(f'Generating PDF plots in `{output_dir}`...')
    
    x, pdf = distribution.pdf()
    plt.plot(x, pdf, label="p(x)")

    # Plot loss function
    x = torch.tensor(np.linspace(-3, 3, 200))
    plt.plot(x, polynomial(x), linestyle='dashed', label="f(x)")

    plt.title(f"PDF of {path.stem}.")
    plt.legend()
    plt.xlabel("x")
    plt.ylim(0, 1)

    plt.show()

Generating images in `results\vae\mnist\MVD\1\11894567\images`...
Generating images in `results\vae\mnist\MVD\1\16403198\images`...
Generating images in `results\vae\mnist\MVD\1\23478912\images`...
Generating images in `results\vae\mnist\MVD\1\56243789\images`...
Generating images in `results\vae\mnist\MVD\1\65738917\images`...
Generating images in `results\vae\mnist\Pathwise\1\11894567\images`...
Generating images in `results\vae\mnist\Pathwise\1\16403198\images`...
Generating images in `results\vae\mnist\Pathwise\1\23478912\images`...
Generating images in `results\vae\mnist\Pathwise\1\56243789\images`...
Generating images in `results\vae\mnist\Pathwise\1\65738917\images`...
Generating images in `results\vae\mnist\Pathwise\80\11894567\images`...
Generating images in `results\vae\mnist\Pathwise\80\16403198\images`...
Generating images in `results\vae\mnist\Pathwise\80\23478912\images`...
Generating images in `results\vae\mnist\Pathwise\80\56243789\images`...
Generating images in `resul